# Esther's Text Processing Notebook Using Spacy

This notebook will focus on using Spacy for natural language processing.

### Importing and Installing

In [12]:
# Run in terminal for importing english pipeline, use of GPU, to train models

!pip install -U pip setuptools wheel
!pip install -U 'spacy[cuda11x,transformers,lookups]'
!python -m spacy download en_core_web_trf


ERROR: Invalid requirement: "'spacy[cuda11x,transformers,lookups]'"
c:\Users\esthe\AppData\Local\Programs\Python\Python311\python.exe: No module named spacy


In [1]:
from tqdm import tqdm

In [2]:

import pandas as pd
import time
import numpy as np
import multiprocessing

### Retrieving the Dataset from Dropbox + Cleaning

In [3]:
import sys
sys.path.append('C:\\Users\\esthe\\DSEI2100_Project\\machine-learning-dse-i210-final-project-fake-news\\data\\external')


In [4]:
import os
os.environ['PYTHONPATH'] = os.path.join(os.environ.get('PYTHONPATH', ''), 'C:\\Users\\esthe\\DSEI2100_Project\\machine-learning-dse-i210-final-project-fake-news\\data\\external')


In [5]:
from config import dropbox_access_token

In [2]:

import dropbox
import pandas as pd
import io
import os 
#from config import dropbox_access_token

dropbox_access_token = ''

# Initialize Dropbox client
dbx = dropbox.Dropbox(dropbox_access_token)

# Dropbox file path
dropbox_file_path = '/fake_real_ML_project_dataset.csv'

# Download the file
metadata, response = dbx.files_download(dropbox_file_path)

# Read the CSV file from the response content using io.BytesIO
df = pd.read_csv(io.BytesIO(response.content))

# Show the results
print("Dataframe loaded from Dropbox:")
print(df.head())

Dataframe loaded from Dropbox:
   Unnamed: 0                                            Content  \
0           0  new york reuters us environmental group sierra...   
1           1  washington reuters us air force asked industry...   
2           2  saturday paul ryan posted photo instagram phot...   
3           3  america keeps waiting word hillary indicted ob...   
4           4                   religion peace ht weasel zippers   

          Title  Type  
0  politicsNews  true  
1  politicsNews  true  
2          News  fake  
3      politics  fake  
4     left-news  fake  


In [ ]:
%run C:\Users\esthe\DSEI2100_Project\machine-learning-dse-i210-final-project-fake-news\data\external\ML_dataset_dropbox.py

In [4]:
df = df.dropna()

In [5]:
# remove extra column
df = df.drop('Unnamed: 0', axis = 1)

In [6]:
df.head()

,Content,Title,Type
0,new york reuters us environmental group sierra...,politicsNews,true
1,washington reuters us air force asked industry...,politicsNews,true
2,saturday paul ryan posted photo instagram phot...,News,fake
3,america keeps waiting word hillary indicted ob...,politics,fake
4,religion peace ht weasel zippers,left-news,fake


In [1]:
df.shape

NameError: name 'df' is not defined

## Using Spacy

This section of the notebook will focus on using Spacy to process and extract information relevant to our goal of creating a fake news classification model.

We will use GPU to do these tasks and time them in order to compare them to our NLTK base.

### Spacy Tokenizer

In [7]:
# load the english model
import spacy
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_trf')

C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [17]:
def tokenize_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens

In [18]:
# tokenize
for i in tqdm(range(100)):
    df['Tokens'] = df['Content'].apply(lambda x: tokenize_text(x))


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [07:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
print(df['Tokens'].head())

### Spacy Lemmatizer

In [1]:
# lemmatizethe text (takes much longer to run than nltk)

for i in tqdm(range(100)):

    df['lem_toks_spcy'] = df['Tokens'].apply(lambda tokens: [nlp(token) for token in tokens])

NameError: name 'df' is not defined

### Spacy POS Tagging

In [9]:
# Define your tag_text function
def tag_text(text):
    doc = nlp(text)
    tags = [(token.text, token.pos_) for token in doc]
    return tags

# Define a function for parallel processing
def apply_parallel(df, func, num_processes):
    df_split = np.array_split(df, num_processes)
    with multiprocessing.Pool(num_processes) as pool:
        df = pd.concat(pool.map(func, df_split))
    return df




In [10]:
# Assuming df is your pandas DataFrame
num_processes = multiprocessing.cpu_count()  # Number of CPU cores
print(num_processes)


8


In [11]:
df['pos_tag'] = apply_parallel(df['Content'], tag_text, num_processes)

C:\Users\esthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [19]:
# create a function to apply spacy tagging to each element in the column
def tag_text(text):
    doc = nlp(text)
    tags = [(token.text, token.pos_) for token in doc]
    return tags

In [20]:
# apply pos tagging to content
for i in tqdm(range(100)):
    df['pos_tag'] = df['Content'].apply(tag_text)

  0%|          | 0/100 [05:26<?, ?it/s]


KeyboardInterrupt: 

### Spacy Parser

### Spacy Entity Recognizer

### Spacy Text Categorizer